In [ ]:
import os

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import statsmodels.api as sm


from prophet import Prophet

In [ ]:
data_folder = os.path.join("..", "..", "data", "berlin")

clean_data_folder = os.path.join(data_folder, "clean_data")
projections_folder = os.path.join(data_folder, "projections")

my_projections_folder = os.path.join(projections_folder, "found_by_me")
cat_projections_folder = os.path.join(projections_folder, "cat")

# Load Clean Data

In [ ]:
surface_df = pd.read_excel(
    os.path.join(clean_data_folder, "surface.xlsx")
)
ground_df = pd.read_excel(
    os.path.join(clean_data_folder, "ground.xlsx")
)

flow_river_projections = pd.read_excel(
    os.path.join(my_projections_folder, "flow_river.xlsx")
)

air_temp_projections = pd.read_excel(
    os.path.join(my_projections_folder, "air_temp.xlsx")
)

precip_projections = pd.read_excel(
    os.path.join(my_projections_folder, "precip.xlsx")
)

cat_flow_river_projections = pd.read_excel(
    os.path.join(cat_projections_folder, "flow_river.xlsx")
)

cat_air_temp_projections = pd.read_excel(
    os.path.join(cat_projections_folder, "air_temp.xlsx")
)

cat_precip_projections = pd.read_excel(
    os.path.join(cat_projections_folder, "precip.xlsx")
)

# Projections Comparison

## Flow River

In [ ]:
column = 'Flow River Rate (m³/s)'

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = flow_river_projections[flow_river_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    # === Trend Computation ===
    
    # 1. Extract trend with Prophet
    
    # 1.1. Historical data
    
    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    hist_forecasting_final = pd.merge(
        forecast,
        df,
        how="inner",
        on="ds",
    )
    
    # 1.2. RCP 4.5
    
    rcp45.index.name = "ds"

    rcp45 = rcp45.reset_index()

    rcp45.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(rcp45)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    rcp45_forecasting_final = pd.merge(
        forecast,
        rcp45,
        how="inner",
        on="ds",
    )
    
    # 1.3. RCP 8.5
    
    rcp85.index.name = "ds"

    rcp85 = rcp85.reset_index()

    rcp85.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(rcp85)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    rcp85_forecasting_final = pd.merge(
        forecast,
        rcp85,
        how="inner",
        on="ds",
    )
    
    # 2. Compute linear regression on the trend
    
    # 2.1. Historical data
    
    X = np.arange(df.shape[0])
    X = sm.add_constant(X)
    y = df["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    hist_line = pd.Series(results.predict(X), index=df['ds'])
    
    hist_slope = results.params.iloc[1]
    hist_p_value = results.pvalues.iloc[1]
    
    # 2.2. RCP 4.5
    
    X = np.arange(rcp45.shape[0])
    X = sm.add_constant(X)
    y = rcp45["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    rcp45_line = pd.Series(results.predict(X), index=rcp45['ds'])
    
    rcp45_slope = results.params.iloc[1]
    rcp45_p_value = results.pvalues.iloc[1]
    
    # 2.3. RCP 8.5
    
    X = np.arange(rcp85.shape[0])
    X = sm.add_constant(X)
    y = rcp85["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    rcp85_line = pd.Series(results.predict(X), index=rcp85['ds'])
    
    rcp85_slope = results.params.iloc[1]
    rcp85_p_value = results.pvalues.iloc[1]
    
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df['ds'],
            y=df['y'],
            mode="lines",
            name="Historical",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45['ds'],
            y=rcp45['y'],
            mode="lines",
            name="RCP 4.5",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85['ds'],
            y=rcp85['y'],
            mode="lines",
            name="RCP 8.5",
        )
    )
    
    
    fig.add_trace(
        go.Scatter(
            x=hist_forecasting_final["ds"],
            y=hist_forecasting_final["trend"],
            mode="lines",
            name="Historical Trend",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45_forecasting_final["ds"],
            y=rcp45_forecasting_final["trend"],
            mode="lines",
            name="RCP 4.5 Trend",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85_forecasting_final["ds"],
            y=rcp85_forecasting_final["trend"],
            mode="lines",
            name="RCP 8.5 Trend",
        )
    )
    
    # add box with regression slope and p-value for all the scenarios
    
    fig.add_annotation(
        x=0.95,
        y=0.99,
        xref="paper",
        yref="paper",
        text=f"Historical - Slope: {hist_slope:.3f}, P-value: {hist_p_value:.2f}",
        showarrow=False,
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.93,
        xref="paper",
        yref="paper",
        text=f"RCP 4.5 - Slope: {rcp45_slope:.3f}, P-value: {rcp45_p_value:.2f}",
        showarrow=False,
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.87,
        xref="paper",
        yref="paper",
        text=f"RCP 8.5 - Slope: {rcp85_slope:.3f}, P-value: {rcp85_p_value:.2f}",
        showarrow=False,
    )
    
    
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = flow_river_projections[flow_river_projections["Station"] == station_id].copy()
    cat_projections_df = cat_flow_river_projections[cat_flow_river_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    cat_rcp45 = cat_projections_df[(cat_projections_df['label'] == "rcp45") & (cat_projections_df['Station'] == station_id)]
    cat_rcp85 = cat_projections_df[(cat_projections_df['label'] == "rcp85") & (cat_projections_df['Station'] == station_id)]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    cat_rcp45.set_index("DateTime", inplace=True)
    cat_rcp85.set_index("DateTime", inplace=True)
    
    
    # plot the data and the projections
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            mode="lines",
            name='Historical',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45.index,
            y=rcp45[column],
            mode="lines",
            name='RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85.index,
            y=rcp85[column],
            mode="lines",
            name='RCP 8.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_rcp45.index,
            y=cat_rcp45[column],
            mode="markers",
            name='Cat RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_rcp85.index,
            y=cat_rcp85[column],
            mode="markers",
            name='Cat RCP 8.5',
        )
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = flow_river_projections[flow_river_projections["Station"] == station_id].copy()
    cat_projections_df = cat_flow_river_projections[cat_flow_river_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    cat_hist = cat_projections_df[cat_projections_df['label'] == "historical"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    cat_hist.set_index("DateTime", inplace=True) 
    
    # take common dates
    common_dates = df.index.intersection(rcp45.index).intersection(rcp85.index).intersection(df.index)  
    
    df = df.loc[common_dates]
    rcp45 = rcp45.loc[common_dates]
    rcp85 = rcp85.loc[common_dates] 
    
    # plot the data and the projections
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            mode="lines",
            name='Historical',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45.index,
            y=rcp45[column],
            mode="lines",
            name='RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85.index,
            y=rcp85[column],
            mode="lines",
            name='RCP 8.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_hist.index,
            y=cat_hist[column],
            mode="markers",
            name='Cat Historical',
        )
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

## Air Temperature

In [ ]:
column = 'Air Temperature (°C)'

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()
    
    station_df.dropna(subset=['Flow River Rate (m³/s)'], inplace=True)

    df = station_df[["DateTime", column]].copy()
    
    my_projection_df = air_temp_projections[air_temp_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    # === Trend Computation ===
    
    # 1. Extract trend with Prophet
    
    # 1.1. Historical data
    
    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    hist_forecasting_final = pd.merge(
        forecast,
        df,
        how="inner",
        on="ds",
    )
    
    # 1.2. RCP 4.5
    
    rcp45.index.name = "ds"

    rcp45 = rcp45.reset_index()

    rcp45.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(rcp45)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    rcp45_forecasting_final = pd.merge(
        forecast,
        rcp45,
        how="inner",
        on="ds",
    )
    
    # 1.3. RCP 8.5
    
    rcp85.index.name = "ds"

    rcp85 = rcp85.reset_index()

    rcp85.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(rcp85)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    rcp85_forecasting_final = pd.merge(
        forecast,
        rcp85,
        how="inner",
        on="ds",
    )
    
    # 2. Compute linear regression on the trend
    
    # 2.1. Historical data
    
    X = np.arange(df.shape[0])
    X = sm.add_constant(X)
    y = df["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    hist_line = pd.Series(results.predict(X), index=df['ds'])
    
    hist_slope = results.params.iloc[1]
    hist_p_value = results.pvalues.iloc[1]
    
    # 2.2. RCP 4.5
    
    X = np.arange(rcp45.shape[0])
    X = sm.add_constant(X)
    y = rcp45["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    rcp45_line = pd.Series(results.predict(X), index=rcp45['ds'])
    
    rcp45_slope = results.params.iloc[1]
    rcp45_p_value = results.pvalues.iloc[1]
    
    # 2.3. RCP 8.5
    
    X = np.arange(rcp85.shape[0])
    X = sm.add_constant(X)
    y = rcp85["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    rcp85_line = pd.Series(results.predict(X), index=rcp85['ds'])
    
    rcp85_slope = results.params.iloc[1]
    rcp85_p_value = results.pvalues.iloc[1]
    
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df['ds'],
            y=df['y'],
            mode="lines",
            name="Historical",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45['ds'],
            y=rcp45['y'],
            mode="lines",
            name="RCP 4.5",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85['ds'],
            y=rcp85['y'],
            mode="lines",
            name="RCP 8.5",
        )
    )
    
    
    fig.add_trace(
        go.Scatter(
            x=hist_forecasting_final["ds"],
            y=hist_forecasting_final["trend"],
            mode="lines",
            name="Historical Trend",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45_forecasting_final["ds"],
            y=rcp45_forecasting_final["trend"],
            mode="lines",
            name="RCP 4.5 Trend",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85_forecasting_final["ds"],
            y=rcp85_forecasting_final["trend"],
            mode="lines",
            name="RCP 8.5 Trend",
        )
    )
    
    # add box with regression slope and p-value for all the scenarios
    
    fig.add_annotation(
        x=0.95,
        y=0.99,
        xref="paper",
        yref="paper",
        text=f"Historical - Slope: {hist_slope:.3f}, P-value: {hist_p_value:.2f}",
        showarrow=False,
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.93,
        xref="paper",
        yref="paper",
        text=f"RCP 4.5 - Slope: {rcp45_slope:.3f}, P-value: {rcp45_p_value:.2f}",
        showarrow=False,
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.87,
        xref="paper",
        yref="paper",
        text=f"RCP 8.5 - Slope: {rcp85_slope:.3f}, P-value: {rcp85_p_value:.2f}",
        showarrow=False,
    )
    
    
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = air_temp_projections[air_temp_projections["Station"] == station_id].copy()
    cat_projections_df = cat_air_temp_projections[cat_air_temp_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    cat_rcp45 = cat_projections_df[cat_projections_df['label'] == "rcp45"]
    cat_rcp85 = cat_projections_df[cat_projections_df['label'] == "rcp85"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    cat_rcp45.set_index("DateTime", inplace=True)
    cat_rcp85.set_index("DateTime", inplace=True)
    
    
    # plot the data and the projections
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            mode="lines",
            name='Historical',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45.index,
            y=rcp45[column],
            mode="lines",
            name='RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85.index,
            y=rcp85[column],
            mode="lines",
            name='RCP 8.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_rcp45.index,
            y=cat_rcp45[column],
            mode="markers",
            name='Cat RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_rcp85.index,
            y=cat_rcp85[column],
            mode="markers",
            name='Cat RCP 8.5',
        )
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = air_temp_projections[air_temp_projections["Station"] == station_id].copy()
    cat_projections_df = cat_air_temp_projections[cat_air_temp_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    cat_hist = cat_projections_df[cat_projections_df['label'] == "historical"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    cat_hist.set_index("DateTime", inplace=True)
    
    # take common dates
    common_dates = df.index.intersection(rcp45.index).intersection(rcp85.index).intersection(df.index)  
    
    df = df.loc[common_dates]
    rcp45 = rcp45.loc[common_dates]
    rcp85 = rcp85.loc[common_dates] 
    
    
    # plot the data and the projections
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            mode="lines",
            name='Historical',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45.index,
            y=rcp45[column],
            mode="lines",
            name='RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85.index,
            y=rcp85[column],
            mode="lines",
            name='RCP 8.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_hist.index,
            y=cat_hist[column],
            mode="markers",
            name='Cat Historical',
        )
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
# Se si fa dal 2000 in poi, si vede che la temperatura aumenta

## Precipitation

In [ ]:
column = 'Cumulated Rainfall (mm)'

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()
    
    station_df.dropna(subset=['Flow River Rate (m³/s)'], inplace=True)

    df = station_df[["DateTime", column]].copy()
    
    my_projection_df = precip_projections[precip_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    # === Trend Computation ===
    
    # 1. Extract trend with Prophet
    
    # 1.1. Historical data
    
    df.index.name = "ds"

    df = df.reset_index()

    df.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(df)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    hist_forecasting_final = pd.merge(
        forecast,
        df,
        how="inner",
        on="ds",
    )
    
    # 1.2. RCP 4.5
    
    rcp45.index.name = "ds"

    rcp45 = rcp45.reset_index()

    rcp45.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(rcp45)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    rcp45_forecasting_final = pd.merge(
        forecast,
        rcp45,
        how="inner",
        on="ds",
    )
    
    # 1.3. RCP 8.5
    
    rcp85.index.name = "ds"

    rcp85 = rcp85.reset_index()

    rcp85.rename(columns={column: "y"}, inplace=True)

    model = Prophet()
    model.fit(rcp85)
    # Make predictions for both columns
    future = model.make_future_dataframe(periods=0)
    forecast = model.predict(future)

    # Merging forecasted data with your original data
    rcp85_forecasting_final = pd.merge(
        forecast,
        rcp85,
        how="inner",
        on="ds",
    )
    
    # 2. Compute linear regression on the trend
    
    # 2.1. Historical data
    
    X = np.arange(df.shape[0])
    X = sm.add_constant(X)
    y = df["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    hist_line = pd.Series(results.predict(X), index=df['ds'])
    
    hist_slope = results.params.iloc[1]
    hist_p_value = results.pvalues.iloc[1]
    
    # 2.2. RCP 4.5
    
    X = np.arange(rcp45.shape[0])
    X = sm.add_constant(X)
    y = rcp45["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    rcp45_line = pd.Series(results.predict(X), index=rcp45['ds'])
    
    rcp45_slope = results.params.iloc[1]
    rcp45_p_value = results.pvalues.iloc[1]
    
    # 2.3. RCP 8.5
    
    X = np.arange(rcp85.shape[0])
    X = sm.add_constant(X)
    y = rcp85["y"].copy()

    model = sm.OLS(y, X)
    results = model.fit()

    # plot the line of the linear regression
    rcp85_line = pd.Series(results.predict(X), index=rcp85['ds'])
    
    rcp85_slope = results.params.iloc[1]
    rcp85_p_value = results.pvalues.iloc[1]
    
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df['ds'],
            y=df['y'],
            mode="lines",
            name="Historical",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45['ds'],
            y=rcp45['y'],
            mode="lines",
            name="RCP 4.5",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85['ds'],
            y=rcp85['y'],
            mode="lines",
            name="RCP 8.5",
        )
    )
    
    
    fig.add_trace(
        go.Scatter(
            x=hist_forecasting_final["ds"],
            y=hist_forecasting_final["trend"],
            mode="lines",
            name="Historical Trend",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45_forecasting_final["ds"],
            y=rcp45_forecasting_final["trend"],
            mode="lines",
            name="RCP 4.5 Trend",
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85_forecasting_final["ds"],
            y=rcp85_forecasting_final["trend"],
            mode="lines",
            name="RCP 8.5 Trend",
        )
    )
    
    # add box with regression slope and p-value for all the scenarios
    
    fig.add_annotation(
        x=0.95,
        y=0.99,
        xref="paper",
        yref="paper",
        text=f"Historical - Slope: {hist_slope:.3f}, P-value: {hist_p_value:.2f}",
        showarrow=False,
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.93,
        xref="paper",
        yref="paper",
        text=f"RCP 4.5 - Slope: {rcp45_slope:.3f}, P-value: {rcp45_p_value:.2f}",
        showarrow=False,
    )
    
    fig.add_annotation(
        x=0.95,
        y=0.87,
        xref="paper",
        yref="paper",
        text=f"RCP 8.5 - Slope: {rcp85_slope:.3f}, P-value: {rcp85_p_value:.2f}",
        showarrow=False,
    )
    
    
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = precip_projections[precip_projections["Station"] == station_id].copy()
    cat_projections_df = cat_precip_projections[cat_precip_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    cat_rcp45 = cat_projections_df[cat_projections_df['label'] == "rcp45"]
    cat_rcp85 = cat_projections_df[cat_projections_df['label'] == "rcp85"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    cat_rcp45.set_index("DateTime", inplace=True)
    cat_rcp85.set_index("DateTime", inplace=True)
    
    
    # plot the data and the projections
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            mode="lines",
            name='Historical',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45.index,
            y=rcp45[column],
            mode="lines",
            name='RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85.index,
            y=rcp85[column],
            mode="lines",
            name='RCP 8.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_rcp45.index,
            y=cat_rcp45[column],
            mode="markers",
            name='Cat RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_rcp85.index,
            y=cat_rcp85[column],
            mode="markers",
            name='Cat RCP 8.5',
        )
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()

In [ ]:
for station_id in surface_df["Station"].unique():
    station_df = surface_df[surface_df["Station"] == station_id].copy()

    df = station_df[["DateTime", column]].copy()
    
    df.dropna(inplace=True)
    
    my_projection_df = precip_projections[precip_projections["Station"] == station_id].copy()
    cat_projections_df = cat_precip_projections[cat_precip_projections["Station"] == station_id].copy()
    
    rcp45 = my_projection_df[my_projection_df["Type"] == "rcp45"]
    rcp85 = my_projection_df[my_projection_df["Type"] == "rcp85"]
    
    cat_hist = cat_projections_df[cat_projections_df['label'] == "historical"]
    
    df.set_index("DateTime", inplace=True)
    
    rcp45.set_index("DateTime", inplace=True)
    rcp85.set_index("DateTime", inplace=True)
    
    cat_hist.set_index("DateTime", inplace=True)
    
    # take common dates
    common_dates = df.index.intersection(rcp45.index).intersection(rcp85.index).intersection(df.index)  
    
    df = df.loc[common_dates]
    rcp45 = rcp45.loc[common_dates]
    rcp85 = rcp85.loc[common_dates] 
    
    # plot the data and the projections
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[column],
            mode="lines",
            name='Historical',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp45.index,
            y=rcp45[column],
            mode="lines",
            name='RCP 4.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=rcp85.index,
            y=rcp85[column],
            mode="lines",
            name='RCP 8.5',
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=cat_hist.index,
            y=cat_hist[column],
            mode="markers",
            name='Cat Historical',
        )
    )
    
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=column,
        font=dict(
            size=18,
        ),
        title=f"{station_id} - {column}",
    )
    
    fig.show()